## Training: Learning Rates

The learning rate is one of the most important hyperparameters when training a neural network. In general, the optimal learning rate is about half of the maximum learning rate (i.e., the learning rate above which the training algorithm diverges - that is, where it's performance begins to suffer). One way to find a good learning rate is to train the model for a few hundred iterations, starting with a very low learning rate (e.g., $10^{–5}$) and gradually increasing it up to a very large value (e.g., 10). This is done by multiplying the learning rate by a constant factor at each iteration (e.g., to go from $10^{–5}$ to 10 in 500 iterations). 

If you plot the loss as a function of the learning rate (using a log scale for the learning rate), you should see it dropping at first. But after a while, the learning rate will be too large, so the loss will shoot back up: the optimal learning rate will be a bit lower than the point at which the loss starts to climb (typically about 10 times lower than the turning point). You can then reinitialize your model and train it normally using this good learning rate.  Here's how to do this in code.

In [3]:
#!pip install tensorflow

  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)


In [2]:
# Using Fashion MNIST as a working example
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

def build_model(seed=42):
    tf.random.set_seed(seed)
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

def build_and_train_model(optimizer):
    model = build_model()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model.fit(X_train, y_train, epochs=10,
                     validation_data=(X_valid, y_valid))

2024-04-09 13:18:17.207934: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 13:18:18.273625: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 13:18:20.869333: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 13:18:24.000389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


**Step 1:  Increasing the learning rate**

The following callback increases the learning rate by a factor in each batch, and also keeps track of the losses.  This is a little kludgey because keras only reports mean loss (over all batches thus far) so we need to keep track of things to figure out how much loss is incurred by each batch. 

In [3]:
import math
K = tf.keras.backend

class ExponentialLearningRate(tf.keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []

    def on_epoch_begin(self, epoch, logs=None):
        self.sum_of_epoch_losses = 0

    def on_batch_end(self, batch, logs=None):
        mean_epoch_loss = logs["loss"]  # the epoch's mean loss so far 
        new_sum_of_epoch_losses = mean_epoch_loss * (batch + 1)
        batch_loss = new_sum_of_epoch_losses - self.sum_of_epoch_losses
        self.sum_of_epoch_losses = new_sum_of_epoch_losses
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))
        self.losses.append(batch_loss)
        K.set_value(self.model.optimizer.learning_rate,
                    self.model.optimizer.learning_rate * self.factor)

**Step 2: Probing the model**

The following routine trains a model using the above callback to update the learning rate through a range of values, and then resets the model when done.

In [4]:
def find_learning_rate(model, X, y, epochs=1, batch_size=32, min_rate=1e-4,
                       max_rate=1):
    init_weights = model.get_weights()
    iterations = math.ceil(len(X) / batch_size) * epochs
    factor = (max_rate / min_rate) ** (1 / iterations)
    init_lr = K.get_value(model.optimizer.learning_rate)
    K.set_value(model.optimizer.learning_rate, min_rate)
    exp_lr = ExponentialLearningRate(factor)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size,
                        callbacks=[exp_lr])
    K.set_value(model.optimizer.learning_rate, init_lr)
    model.set_weights(init_weights)
    return exp_lr.rates, exp_lr.losses

**Step 3: Inspecting the data**

Finally, we simply plot the losses against the learning rate to identify a "peak" and then we'll back off from there.

In [5]:
import matplotlib.pyplot as plt

def plot_lr_vs_loss(rates, losses):
    plt.plot(rates, losses, "b")
    plt.gca().set_xscale('log')
    max_loss = losses[0] + min(losses)
    plt.hlines(min(losses), min(rates), max(rates), color="k")
    plt.axis([min(rates), max(rates), 0, max_loss])
    plt.xlabel("Learning rate")
    plt.ylabel("Loss")
    plt.grid()

Now we build the model and find the optimal learning rate.

In [6]:
model = build_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=["accuracy"])
batch_size = 128
rates, losses = find_learning_rate(model, X_train, y_train, epochs=1,
                                   batch_size=batch_size)
plot_lr_vs_loss(rates, losses)

/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


AttributeError: 'str' object has no attribute 'name'

This suggests an optimal learning rate around $ 10^{-1} $.  So, let's give it a try.

In [7]:
optimal_lr = 0.1
optimizer = tf.keras.optimizers.SGD(learning_rate=optimal_lr)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,metrics=["accuracy"])
history_optimal_lr = model.fit(X_train, y_train, epochs=10,
                     validation_data=(X_valid, y_valid))

Epoch 1/10


2024-04-09 13:21:54.025864: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7251 - loss: 0.7513 - val_accuracy: 0.8326 - val_loss: 0.4508
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8477 - loss: 0.4155 - val_accuracy: 0.8514 - val_loss: 0.3899
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8666 - loss: 0.3661 - val_accuracy: 0.8548 - val_loss: 0.3770
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8749 - loss: 0.3376 - val_accuracy: 0.8628 - val_loss: 0.3648
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8817 - loss: 0.3146 - val_accuracy: 0.8644 - val_loss: 0.3713
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8884 - loss: 0.2992 - val_accuracy: 0.8616 - val_loss: 0.3753
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8922 - loss: 0.2861 - val_accuracy: 0.8752 - val_loss: 0.3496
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8968 - loss: 0.2733 - val_accurac

### Exercise

Verify that 0.1 is a better learning rate than something much smaller (e.g. .001) or much larger (e.g., .5).

### Learning Rate Schedules

A constant learning rate doesn't account for the possibility that the gradient landscape changes as we move across it.  Optimizers can help address this, but it is also possible to adjust the learning rate according to a schedule.  In the following, we discuss a few approaches for this.

#### Motivation?

1. **Early Training:** Initially, a larger learning rate can be beneficial, enabling rapid learning and helping the model to quickly escape suboptimal local minima.
2. **Later Stages:** As training progresses, reducing the learning rate helps to stabilize the learning process and fine-tune the model parameters, leading to better convergence.
3. **Preventing Overfitting and Oscillations:** A carefully chosen learning rate schedule can prevent the model from overfitting and reduce oscillations near the minima.

<div style="text-align: center">

![learning rate](assets/learning_rate.png)
</div>

Learning rate scheduling is thus a balancing act - starting with a higher learning rate for faster convergence initially, then decreasing it to allow more fine-grained adjustments as the model starts converging.

### Methods

1. **Power Scheduling:**
   - **Description:** Gradually decreases the learning rate over time using a polynomial decay, typically quadratic.
   - **Formulation:** $ \eta(t) = \eta_0 / (1 + t/s)^c $, where $ \eta_0 $ is the initial learning rate, $ t $ is the iteration number, $ s $ is a step parameter, and $ c $ is the power, often set to 1.


In [8]:
# Keras supports learning rate scheduling by extending the LearningRateSchedule class, and then passing that to a optimizer.  To implement a PowerSchedule, we can do the following:

class PowerScheduling(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_rate, step_param):
        super(PowerScheduling, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_rate = decay_rate
        self.step_param = step_param

    def __call__(self, step):
        # Note the "cast" here - necessary to make sure we get the correct type back
        return self.initial_learning_rate / (1 + tf.cast(step, tf.float32) / self.step_param) ** self.decay_rate


# Example usage
initial_learning_rate = 0.1
decay_rate = 1.0  # For linear decay
step_param = 1000.0  # Determines how quickly the learning rate decreases

lr_schedule = PowerScheduling(initial_learning_rate, decay_rate, step_param)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
history_power_scheduling = build_and_train_model(optimizer)

Epoch 1/10


2024-04-09 13:22:26.052289: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7329 - loss: 0.7297 - val_accuracy: 0.8366 - val_loss: 0.4269
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8573 - loss: 0.3915 - val_accuracy: 0.8548 - val_loss: 0.3844
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8708 - loss: 0.3498 - val_accuracy: 0.8646 - val_loss: 0.3662
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8799 - loss: 0.3266 - val_accuracy: 0.8626 - val_loss: 0.3563
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8846 - loss: 0.3121 - val_accuracy: 0.8660 - val_loss: 0.3504
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8880 - loss: 0.3019 - val_accuracy: 0.8676 - val_loss: 0.3442
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8914 - loss: 0.2936 - val_accuracy: 0.8694 - val_loss: 0.3413
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8936 - loss: 0.2871 - val_accurac


2. **Exponential Scheduling:**
   - **Description:** Cuts the learning rate by a factor of $ \gamma $ every $ s $ steps.
   - **Formulation:** $ \eta(t) = \eta_0 \cdot \gamma^{t/s} $.
   - **Senior et al's Recommendation:** Particularly recommended for its effectiveness and simplicity in many applications.


In [9]:
import math

# First, we'll calculate a learning rate based on the number of batches we anticipate
n_epochs = 10
batch_size = 32
n_steps = n_epochs * math.ceil(len(X_train) / batch_size)

lr0=.1

# Keras now has an ExponentialDecay scheduling class we can use here
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr0,
                                                             decay_steps=n_steps,
                                                             decay_rate=.95)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
history_exponential_scheduling = build_and_train_model(optimizer)

Epoch 1/10


2024-04-09 13:25:59.948074: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7276 - loss: 0.7441 - val_accuracy: 0.8138 - val_loss: 0.4863
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8491 - loss: 0.4119 - val_accuracy: 0.8340 - val_loss: 0.4316
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8664 - loss: 0.3630 - val_accuracy: 0.8484 - val_loss: 0.4117
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8756 - loss: 0.3349 - val_accuracy: 0.8604 - val_loss: 0.3839
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8825 - loss: 0.3153 - val_accuracy: 0.8626 - val_loss: 0.3794
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8867 - loss: 0.2982 - val_accuracy: 0.8656 - val_loss: 0.3754
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8943 - loss: 0.2853 - val_accuracy: 0.8582 - val_loss: 0.4007
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8980 - loss: 0.2737 - val_accurac


3. **Piecewise Constant Scheduling:**
   - **Description:** Uses a constant learning rate for a set number of epochs and then lowers it to another constant rate for another set of epochs, and so on.
   - **Formulation:** Implementing a pre-defined series of constant learning rates at different training stages.


In [10]:
# Let's say we want a learning rate of .1 for the first 3 epochs, .05 for the next three, and then .005 for the remainder.  So, we'll calculate the number of batches per epoch given our data.
epoch_steps = math.ceil(len(X_train) / batch_size)
b1 = epoch_steps * 3
b2 = b1+epoch_steps * 3


boundaries = [b1, b2]
values = [.1, 0.05, 0.005]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
history_piecewise = build_and_train_model(optimizer)

Epoch 1/10


2024-04-09 13:26:50.369596: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7264 - loss: 0.7429 - val_accuracy: 0.8310 - val_loss: 0.4340
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8490 - loss: 0.4121 - val_accuracy: 0.8450 - val_loss: 0.4024
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8671 - loss: 0.3663 - val_accuracy: 0.8612 - val_loss: 0.3700
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8851 - loss: 0.3104 - val_accuracy: 0.8726 - val_loss: 0.3430
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8920 - loss: 0.2926 - val_accuracy: 0.8746 - val_loss: 0.3441
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8963 - loss: 0.2805 - val_accuracy: 0.8744 - val_loss: 0.3443
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9045 - loss: 0.2586 - val_accuracy: 0.8844 - val_loss: 0.3164
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9085 - loss: 0.2475 - val_accurac


4. **Performance Scheduling:**
   - **Description:** Measures the validation error every $ N $ steps (or epochs) and reduces the learning rate by $ \lambda $ if the error has stopped dropping.
   - **Senior et al's Recommendation:** Advised for its adaptability to the model’s actual progress.


In [13]:
# Performance based scheduling requires the use of a callback, so we can evaluate the performance of the network
# We can do it like this
lr0 = .01
model = build_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=lr0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)
history_performance_scheduling = model.fit(X_train, y_train, epochs=25,
                    validation_data=(X_valid, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/25
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6757 - loss: 0.9573 - val_accuracy: 0.8278 - val_loss: 0.4823 - learning_rate: 0.0100
Epoch 2/25
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8280 - loss: 0.4898 - val_accuracy: 0.8412 - val_loss: 0.4391 - learning_rate: 0.0100
Epoch 3/25
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8472 - loss: 0.4333 - val_accuracy: 0.8500 - val_loss: 0.4136 - learning_rate: 0.0100
Epoch 4/25
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8579 - loss: 0.4004 - val_accuracy: 0.8532 - val_loss: 0.3979 - learning_rate: 0.0100
Epoch 5/25
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8654 - loss: 0.3772 - val_accuracy: 0.8564 - val_loss: 0.3873 - learning_rate: 0.0100
Epoch 6/25
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8707 - loss: 0.3598 - val_accuracy: 0.8598 - val_loss: 0.3755 - learning_rate: 0.0100
Epoch 7/25
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8744 - l

In [14]:
import matplotlib.pyplot as plt

plt.plot(history_performance_scheduling.epoch, history_performance_scheduling.history["lr"], "bo-")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate", color='b')
plt.tick_params('y', colors='b')
plt.gca().set_xlim(0, 24)
plt.grid(True)

ax2 = plt.gca().twinx()
ax2.plot(history_performance_scheduling.epoch, history_performance_scheduling.history["val_loss"], "r^-")
ax2.set_ylabel('Validation Loss', color='r')
ax2.tick_params('y', colors='r')

plt.title("Reduce LR on Plateau", fontsize=14)
plt.show()

KeyError: 'lr'


5. **1cycle Scheduling:**
   - **Description:** Briefly increases the learning rate halfway through training before decreasing it. Proposed by [Leslie N. Smith in 2018](https://arxiv.org/abs/1803.09820).
   - **Benefits (Smith, 2018):** Shown to lead to faster convergence and better performance. It allows the model to explore a wider range of parameters due to the higher learning rate in the middle of training.
   - **Formulation:** Starts with a low learning rate, increases it linearly for the first half of training, and then decreases it linearly for the second half, possibly followed by a few epochs with a very small learning rate to finalize training.


Implementing the 1-cycle learning rate scheduler in Keras is not hard, but it requires us to obtain an optimal learning rate first before creating a scheduler.  We did this above.  Here, I'm implementing the LR scheduler as a callback due to some of the complexities of working with tensors.

In [15]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None,
                 last_iterations=None, last_lr=None):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr,
                                   self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)

In [16]:
model = build_model()
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(),
              metrics=["accuracy"])
n_epochs = 25
onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * n_epochs,
                             max_lr=0.1)
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid, y_valid),
                    callbacks=[onecycle])

Epoch 1/25


AttributeError: 'str' object has no attribute 'name'



Learning rate scheduling is a powerful tool in the training of neural networks. Different scheduling methods offer various advantages, and the choice of method can depend on the specific characteristics of the task, the dataset, and the desired training dynamics. While 1cycle scheduling has shown promising results for faster convergence, the recommendations of Senior et al. for performance or exponential scheduling highlight the importance of choosing a method that aligns with the model’s learning progress and the practitioner’s familiarity with the training dynamics.